# Pipeline Hola Cetáceos
---
### Giovanny Alejandro Cuervo Londoño

En este notebook se pretende realizar el entubamiento del conjunto de datos de audios de la <a href="https://cis.whoi.edu/science/B/whalesounds/index.cfm\"> Watkins Marine Mammal Sound Database 


In [1]:
import numpy as np
import pandas as pd
import os
import librosa
import re
import tensorflow as tf

In [2]:
!python --version
print('NumPy', np.__version__)
print('Pandas', pd.__version__)
print('librosa', librosa.__version__)
print('re', re.__version__)
print('tensorflow', tf.__version__)

Python 3.8.5
NumPy 1.19.2
Pandas 1.1.3
librosa 0.8.0
re 2.2.1
tensorflow 2.4.1


Este notebook se creó con estas versiones de módulos:
> Python 3.7.10

> NumPy 1.19.5

> Pandas 1.1.5

> librosa 0.8.0

> re 2.2.1

> tensorflow 2.4.1

Conectamos nuestro directorio a de audios

In [3]:
#si esta trabajando en google drive
#from google.colab import drive
#drive.mount('/content/drive')

#Cargamos los datos de un directorio local
pathdir = '/home/mofoko/Desktop/Diplomato intel artificial/IA y AP/Proyecto/Audios/Audios/'
listdir = os.listdir(pathdir)
print(f'Cantidad de audios: {len(listdir)}')

Cantidad de audios: 1697


## Creacion del vector de Etiquetas
---

In [4]:
#funcion para generar las etiquetas
def etiquetas_audios(list_audios):
    
  """
  Esta función tiene como input la lista
  de los nombres de los audios y retorna
  los nombres sin su numero y .mp4
  """
  names = []
  for audio in list_audios:
    split_name = re.split(r'\d', audio, maxsplit=2)
    names.append(split_name[0])
  return names

In [5]:
nombre_cetace = etiquetas_audios(listdir)

## Creación de matriz de audios
---

In [6]:
#convertimos los audios a una matriz shape(109620,2)
def audio2matriz(cetaceo, filas):
  """
  Esta función tiene como input el audio y 
  la cantidad de filas deseadas para retornar
  una matriz de shape(filas, 2) donde las
  dos columansa son amplitud y timepo
  """
  y_1, sr_1 = librosa.load(pathdir+cetaceo)
  duration = len(y_1)/sr_1
  sr = filas/duration 
  y_1 = librosa.resample(y_1, sr_1, sr)
  #algunas veces los decimales no alcanzan y sale con una fila mas
  if len(y_1) == filas+1:
    y_1 = y_1[:filas]
  #convierte amplitudes a tiempos
  amplitu2time = lambda y_1, sr : list(map(lambda x : x/sr,range(len(y_1))))
  time = amplitu2time(y_1, sr)
  #concatena amplitudes y tiempos
  Ampli0time = lambda A, t: np.concatenate((np.array(A).reshape(-1, 1), np.array(t).reshape(-1, 1)), axis= 1)
  Matriz = Ampli0time(y_1, time)
  
  return Matriz



## Creación del Tensor de Audios
---
Se busca crear matrices para cada uno de los audios con el mismo tamaño para poder apilarlos para formar un tensor de dimensiones: (1602, 109620, 2). dos columnas que representan los puntos de amplitudes y el tiempos de 109620 filas de sampleos para 1602 audios.

In [7]:
#definimos un generador
def tupla(n):
  """
  Esta función declara un generador que tiene como input n
  el numero de audios de la lista listdir a generar 
  retornando una tupla de dos: nombre y la matriz del audio 
  """
  listdir_n = listdir[:n]
  list_names = etiquetas_audios(listdir_n)
  for name, path_name in zip(list_names, listdir_n):
    M = audio2matriz(path_name, 109620)
    yield name, M

Creamos nuestro de tensorflow dataset

In [8]:
ds_counter = tf.data.Dataset.from_generator(tupla, args=[3], output_types=(tf.string, tf.float64), output_shapes=((), (109620, 2)))
#args=[3] es el argumento n del generador tupla(3)

In [9]:
it = iter(ds_counter)
print(next(it)) #1er slice
print(next(it)) #2do slice
print(next(it)) #3er slice

(<tf.Tensor: shape=(), dtype=string, numpy=b'Long-Finned Pilot Whale'>, <tf.Tensor: shape=(109620, 2), dtype=float64, numpy=
array([[ 2.66532600e-03,  0.00000000e+00],
       [ 3.46068270e-03,  9.73968618e-06],
       [ 4.04257560e-03,  1.94793724e-05],
       ...,
       [-1.12582243e-03,  1.06763518e+00],
       [-1.19674951e-03,  1.06764492e+00],
       [ 0.00000000e+00,  1.06765466e+00]])>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'False Killer Whale'>, <tf.Tensor: shape=(109620, 2), dtype=float64, numpy=
array([[2.74948007e-03, 0.00000000e+00],
       [3.08781373e-03, 6.52842783e-06],
       [3.29862838e-03, 1.30568557e-05],
       ...,
       [4.68908547e-04, 7.15626673e-01],
       [3.91415844e-04, 7.15633202e-01],
       [2.61516601e-04, 7.15639730e-01]])>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Bowhead Whale'>, <tf.Tensor: shape=(109620, 2), dtype=float64, numpy=
array([[-9.10087526e-02,  0.00000000e+00],
       [-1.84698969e-01,  4.13245758e-04],
       [-7.42307380e